In [2]:
import pdftotext
 
file = open("D:\\판결문\\울산지방법원_2019노83.pdf", 'rb')
fileReader = pdftotext.PDF(file)

ModuleNotFoundError: No module named 'pdftotext'

In [10]:
import PyPDF2
 
file = open("D:\\판결문\\의정부지방법원_2018노3606.pdf", 'rb')
fileReader = PyPDF2.PdfFileReader(file)

 
# 문서의 정보를 읽어드린다
fileReader.documentInfo
 
# 전체 페이지수를 출력한다
print(fileReader.numPages)
 
# 첫 번째 페이지 정보를 가져온다
pageObj = fileReader.getPage(0)
 
# 페이지 정보의 텍스트를 가져온다
text = pageObj.extractText()
text

5


"˙â˚J˙Z˝¦'ı˚I˚Î1−ˆ1&a$õ*5ˆ)#m2ù0Vˆ2$¡ˇµ$F*É#ž2Ù˜)˚Iˆ]3Õ$m ›$õ%²˙˛']ˇÆˆ2$m#.']$m˚½%²˚½˛ƒ$m ›$á0©2ù2Ù$Ý˘Ñ&a˘Ñ&a$1'] y2ù˙þ˙ª)÷˙þ˙ª˙˚˚J"

In [13]:
def async_detect_document(gcs_source_uri, gcs_destination_uri):
    """OCR with PDF/TIFF as source files on GCS"""
    import re
    from google.cloud import vision
    from google.cloud import storage
    from google.protobuf import json_format
    # Supported mime_types are: 'application/pdf' and 'image/tiff'
    mime_type = "application/pdf"

    # How many pages should be grouped into each json output file.
    batch_size = 2

    client = vision.ImageAnnotatorClient()

    feature = vision.types.Feature(
        type=vision.enums.Feature.Type.DOCUMENT_TEXT_DETECTION)

    gcs_source = vision.types.GcsSource(uri=gcs_source_uri)
    input_config = vision.types.InputConfig(
        gcs_source=gcs_source, mime_type=mime_type)

    gcs_destination = vision.types.GcsDestination(uri=gcs_destination_uri)
    output_config = vision.types.OutputConfig(
        gcs_destination=gcs_destination, batch_size=batch_size)

    async_request = vision.types.AsyncAnnotateFileRequest(
        features=[feature], input_config=input_config,
        output_config=output_config)

    operation = client.async_batch_annotate_files(
        requests=[async_request])

    print('Waiting for the operation to finish.')
    operation.result(timeout=420)

    # Once the request has completed and the output has been
    # written to GCS, we can list all the output files.
    storage_client = storage.Client()

    match = re.match(r'gs://([^/]+)/(.+)', gcs_destination_uri)
    bucket_name = match.group(1)
    prefix = match.group(2)

    bucket = storage_client.get_bucket(bucket_name)

    # List objects with the given prefix.
    blob_list = list(bucket.list_blobs(prefix=prefix))
    print('Output files:')
    for blob in blob_list:
        print(blob.name)

    # Process the first output file from GCS.
    # Since we specified batch_size=2, the first response contains
    # the first two pages of the input file.
    output = blob_list[0]

    json_string = output.download_as_string()
    response = json_format.Parse(
        json_string, vision.types.AnnotateFileResponse())

    # The actual response for the first page of the input file.
    first_page_response = response.responses[0]
    annotation = first_page_response.full_text_annotation

    # Here we print the full text from the first page.
    # The response contains more information:
    # annotation/pages/blocks/paragraphs/words/symbols
    # including confidence scores and bounding boxes
    print(u'Full text:\n{}'.format(annotation.text))

In [ ]:
async_detect_document()